# Demo 3: Random Forest and DataStax Analytics
------
<img src="images/drinkWine.jpeg" width="300" height="500">


#### Dataset: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

## What are we trying to learn from this dataset? 

# QUESTION:  Can Random Forest be used to classify if a wine is a good wine (score 9+) by its attributes?

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import random
from random import randint, randrange
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Helper function to have nicer formatting of Spark DataFrames

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

# DataStax Enterprise Analytics
<img src="images/dselogo.png" width="400" height="200">

## Creating Tables and Loading Tables

### Connect to DSE Analytics Cluster

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.01'])
session = cluster.connect()

### Create Demo Keyspace 

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [6]:
session.set_keyspace('accelerate')

### Create table called `wines`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This will have two datasets "white" and "red"

In [7]:
query = "CREATE TABLE IF NOT EXISTS wines \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### Create table called `wineWhite`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This is only for the white wines.

In [8]:
query = "CREATE TABLE IF NOT EXISTS wineWhite \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### Create table called `wineRed`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This is only for the red wines.

In [9]:
query = "CREATE TABLE IF NOT EXISTS wineRed \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### What do these of these 12 columns represent: 

* **Fixed acidity**
* **Volatile acidity**
* **Citric Acid**
* **Residual Sugar** 
* **Chlorides**
* **Free sulfur dioxide**     
* **Total sulfur dioxide**
* **Density** 
* **pH**
* **Sulphates**
* **Alcohol**
* **Quality**

### Load 2 Wine Dataset -- White and Red
<img src="images/whiteAndRed.jpeg" width="300" height="300">

### Load Wine datasets from CSV file (winequality-red.csv winequality-white.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wines`

In [10]:
fileName = 'data/winequality-red.csv'
input_file = open(fileName, 'r')
i = 1
for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

fileName = 'data/winequality-white.csv'
input_file = open(fileName, 'r')

for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1
    

### Load Wine datasets from CSV file (winequality-red.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wineRed`

In [11]:
fileName = 'data/winequality-red.csv'
input_file = open(fileName, 'r')
i = 1
for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wineRed (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

### Load Wine datasets from CSV file (winequality-white.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wineWhite`

In [12]:
fileName = 'data/winequality-white.csv'
input_file = open(fileName, 'r')

for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wineWhite (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

## Machine Learning with DSE Analytics and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to DSE. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [13]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()


wineDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="wines", keyspace="accelerate").load()

print ("Table Row Count: ")
print (wineDF.count())

Table Row Count: 
6497


In [14]:
wineWhiteDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="winewhite", keyspace="accelerate").load()

print ("Table Row Count: ")
print (wineWhiteDF.count())

wineRedDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="winered", keyspace="accelerate").load()

print ("Table Row Count: ")
print (wineRedDF.count())

Table Row Count: 
4898
Table Row Count: 
1599


In [15]:
showDF(wineDF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,3088,9.5,0.044,0.74,0.99720,6.5,68.0,3.18,6.0,13.3,0.54,224.0,0.260
1,6395,10.7,0.035,0.29,0.99142,6.4,44.0,3.17,7.0,1.1,0.55,140.0,0.105
2,381,9.4,0.080,0.42,0.99740,8.3,11.0,3.21,6.0,2.0,0.80,27.0,0.260
3,3638,9.0,0.029,0.27,0.99490,5.5,22.0,3.34,5.0,4.6,0.44,104.0,0.140
4,4845,9.8,0.059,0.29,0.99328,7.6,37.0,3.09,5.0,2.5,0.37,115.0,0.270


In [16]:
wine6DF = wineDF.filter("quality > 5")
showDF(wine6DF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,3088,9.5,0.044,0.74,0.99720,6.5,68.0,3.18,6.0,13.3,0.54,224.0,0.260
1,6395,10.7,0.035,0.29,0.99142,6.4,44.0,3.17,7.0,1.1,0.55,140.0,0.105
2,381,9.4,0.080,0.42,0.99740,8.3,11.0,3.21,6.0,2.0,0.80,27.0,0.260
3,4529,9.1,0.046,0.29,0.99834,6.6,39.0,3.05,6.0,14.4,0.50,118.0,0.220
4,4319,10.9,0.029,0.31,0.99276,7.5,53.0,3.03,6.0,6.5,0.38,160.0,0.180


In [17]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wine6DF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,3088,9.5,0.044,0.74,0.99720,6.5,68.0,3.18,6.0,13.3,0.54,224.0,0.260,"[9.5, 0.04399999976158142, 0.7400000095367432,...",0.0
1,6395,10.7,0.035,0.29,0.99142,6.4,44.0,3.17,7.0,1.1,0.55,140.0,0.105,"[10.699999809265137, 0.03500000014901161, 0.28...",1.0
2,381,9.4,0.080,0.42,0.99740,8.3,11.0,3.21,6.0,2.0,0.80,27.0,0.260,"[9.399999618530273, 0.07999999821186066, 0.419...",0.0
3,4529,9.1,0.046,0.29,0.99834,6.6,39.0,3.05,6.0,14.4,0.50,118.0,0.220,"[9.100000381469727, 0.04600000008940697, 0.289...",0.0
4,4319,10.9,0.029,0.31,0.99276,7.5,53.0,3.03,6.0,6.5,0.38,160.0,0.180,"[10.899999618530273, 0.028999999165534973, 0.3...",0.0


4113


In [18]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Datafram Row Count: ")
print (test.count())

Train Dataframe Row Count: 
3359
Test Datafram Row Count: 
747


In [19]:
!date

Tue May 14 14:02:35 PDT 2019


In [20]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

752


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,9,9.5,0.073,0.02,0.9968,7.8,9.0,3.36,7.0,2.0,0.57,18.0,0.58,"[9.5, 0.0729999989271164, 0.019999999552965164...",1.0,"[9.020345773058336, 0.9024088048482443, 0.0751...","[0.9020345773058335, 0.09024088048482441, 0.00...",0.0
1,17,10.5,0.092,0.56,0.9969,8.5,35.0,3.30,7.0,1.8,0.75,103.0,0.28,"[10.5, 0.09200000017881393, 0.5600000023841858...",1.0,"[7.735058117918804, 2.048309049404317, 0.20788...","[0.7735058117918805, 0.2048309049404317, 0.020...",0.0
2,21,9.4,0.077,0.48,0.9968,8.9,29.0,3.39,6.0,1.8,0.53,60.0,0.22,"[9.399999618530273, 0.07699999958276749, 0.479...",0.0,"[8.59916772132543, 1.297720494389207, 0.096774...","[0.859916772132543, 0.1297720494389207, 0.0096...",0.0
3,109,9.6,0.091,0.53,0.9976,8.0,18.0,3.37,6.0,2.5,0.80,80.0,0.33,"[9.600000381469727, 0.09099999815225601, 0.529...",0.0,"[8.499324461137341, 1.3693505884330939, 0.1204...","[0.8499324461137341, 0.1369350588433094, 0.012...",0.0
4,270,10.1,0.104,0.51,0.9996,11.5,4.0,3.28,6.0,4.0,0.97,23.0,0.18,"[10.100000381469727, 0.10400000214576721, 0.50...",0.0,"[6.61747762152296, 3.115625249838145, 0.261242...","[0.661747762152296, 0.3115625249838145, 0.0261...",0.0


In [21]:
!date

Tue May 14 14:02:39 PDT 2019


In [22]:
showDF(predictions.select("quality", "label", "prediction", "probability"))

,quality,label,prediction,probability
0,7.0,1.0,0.0,"[0.8906998708452006, 0.10076221544993018, 0.00..."
1,6.0,0.0,0.0,"[0.8952671785375083, 0.09753419347190817, 0.00..."
2,6.0,0.0,0.0,"[0.8591317267007845, 0.13443665276865752, 0.00..."
3,7.0,1.0,0.0,"[0.8240638668748117, 0.16942784793927707, 0.00..."
4,6.0,0.0,0.0,"[0.8680211659129998, 0.11896951063316855, 0.01..."


### We can now use the MutliclassClassifciationEvaluator to evalute the accurancy of our predictions. 

In [23]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7058823529411765


In [24]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wineRedDF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,1584,9.8,0.074,0.29,0.99578,6.2,32.0,3.33,5.0,2.1,0.62,98.0,0.46,"[9.800000190734863, 0.07400000095367432, 0.289...",0.0
1,769,9.7,0.082,0.02,0.99744,7.1,24.0,3.55,6.0,2.3,0.53,94.0,0.59,"[9.699999809265137, 0.0820000022649765, 0.0199...",1.0
2,1580,11.3,0.053,0.09,0.99402,6.2,24.0,3.54,5.0,1.7,0.60,32.0,0.56,"[11.300000190734863, 0.05299999937415123, 0.09...",0.0
3,1199,10.9,0.052,0.26,0.99510,7.7,19.0,3.15,6.0,2.0,0.79,77.0,0.26,"[10.899999618530273, 0.052000001072883606, 0.2...",1.0
4,23,9.5,0.106,0.21,0.99660,7.9,10.0,3.17,5.0,1.6,0.91,37.0,0.43,"[9.5, 0.10599999874830246, 0.20999999344348907...",0.0


1599


In [25]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Datafram Row Count: ")
print (test.count())

Train Dataframe Row Count: 
1333
Test Datafram Row Count: 
265


In [26]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

267


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,3,9.8,0.092,0.04,0.9970,7.8,15.0,3.26,5.0,2.3,0.65,54.0,0.76,"[9.800000190734863, 0.09200000017881393, 0.039...",0.0,"[6.345045389871097, 3.035477522855763, 0.30124...","[0.6345045389871097, 0.3035477522855763, 0.030...",0.0
1,9,9.5,0.073,0.02,0.9968,7.8,9.0,3.36,7.0,2.0,0.57,18.0,0.58,"[9.5, 0.0729999989271164, 0.019999999552965164...",2.0,"[4.712474487794458, 4.389986952927961, 0.46359...","[0.47124744877944585, 0.4389986952927961, 0.04...",0.0
2,12,10.5,0.071,0.36,0.9978,7.5,17.0,3.35,5.0,6.1,0.80,102.0,0.50,"[10.5, 0.07100000232458115, 0.3600000143051147...",0.0,"[7.35756280656401, 2.0209773884788853, 0.44782...","[0.735756280656401, 0.20209773884788854, 0.044...",0.0
3,35,9.2,0.103,0.25,0.9957,5.2,13.0,3.38,5.0,1.8,0.55,50.0,0.32,"[9.199999809265137, 0.10300000011920929, 0.25,...",0.0,"[5.7191101321926325, 3.112672649569958, 0.5464...","[0.5719110132192632, 0.31126726495699575, 0.05...",0.0
4,87,9.9,0.110,0.28,0.9972,8.6,20.0,2.93,6.0,1.9,1.95,136.0,0.49,"[9.899999618530273, 0.10999999940395355, 0.280...",1.0,"[5.9654607320954165, 3.7905431786124657, 0.127...","[0.5965460732095417, 0.37905431786124655, 0.01...",0.0


In [27]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6179775280898876


In [28]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wineWhiteDF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,3088,9.5,0.044,0.74,0.99720,6.5,68.0,3.18,6.0,13.3,0.54,224.0,0.260,"[9.5, 0.04399999976158142, 0.7400000095367432,...",0.0
1,6395,10.7,0.035,0.29,0.99142,6.4,44.0,3.17,7.0,1.1,0.55,140.0,0.105,"[10.699999809265137, 0.03500000014901161, 0.28...",2.0
2,3638,9.0,0.029,0.27,0.99490,5.5,22.0,3.34,5.0,4.6,0.44,104.0,0.140,"[9.0, 0.028999999165534973, 0.2700000107288360...",1.0
3,4845,9.8,0.059,0.29,0.99328,7.6,37.0,3.09,5.0,2.5,0.37,115.0,0.270,"[9.800000190734863, 0.05900000035762787, 0.289...",1.0
4,4529,9.1,0.046,0.29,0.99834,6.6,39.0,3.05,6.0,14.4,0.50,118.0,0.220,"[9.100000381469727, 0.04600000008940697, 0.289...",0.0


4898


In [29]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Datafram Row Count: ")
print (test.count())

Train Dataframe Row Count: 
4008
Test Datafram Row Count: 
895


In [30]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

890


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,1603,9.9,0.058,0.32,0.9956,7.2,47.0,3.19,6.0,8.500000,0.40,186.0,0.23,"[9.899999618530273, 0.057999998331069946, 0.31...",0.0,"[5.81893918331252, 2.6832605037656205, 1.05356...","[0.581893918331252, 0.26832605037656204, 0.105...",0.0
1,1604,9.9,0.058,0.32,0.9956,7.2,47.0,3.19,6.0,8.500000,0.40,186.0,0.23,"[9.899999618530273, 0.057999998331069946, 0.31...",0.0,"[5.81893918331252, 2.6832605037656205, 1.05356...","[0.581893918331252, 0.26832605037656204, 0.105...",0.0
2,1611,9.7,0.035,0.40,0.9947,8.6,17.0,3.14,5.0,4.200000,0.53,109.0,0.23,"[9.699999809265137, 0.03500000014901161, 0.400...",1.0,"[5.058546404194094, 3.445548872346555, 1.08179...","[0.5058546404194093, 0.34455488723465544, 0.10...",0.0
3,1639,8.6,0.057,0.39,0.9999,7.3,45.0,3.21,5.0,17.950001,0.36,149.0,0.24,"[8.600000381469727, 0.05700000002980232, 0.389...",1.0,"[3.873672665423615, 4.961275342196807, 0.79718...","[0.38736726654236153, 0.4961275342196808, 0.07...",1.0
4,1683,9.5,0.047,0.30,0.9966,7.3,42.0,3.33,6.0,8.200000,0.46,207.0,0.22,"[9.5, 0.04699999839067459, 0.30000001192092896...",0.0,"[5.002661324060535, 3.5638637030864193, 0.9455...","[0.5002661324060536, 0.356386370308642, 0.0945...",0.0


In [31]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.5531914893617021


In [32]:
session.execute("""drop table wines""")
session.execute("""drop table wineRed""")
session.execute("""drop table wineWhite""")